In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import copy
from tqdm.notebook import tqdm

from sklearn.model_selection import train_test_split
from collections import Counter
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch import optim
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import (
    accuracy_score,
    f1_score,
    classification_report,
    jaccard_score
)

from transformers import (
    T5Tokenizer,
    T5Model,
    T5ForConditionalGeneration,
    get_linear_schedule_with_warmup
)

In [2]:
from google.colab import drive
drive.mount("/content/drive/")

Mounted at /content/drive/


In [8]:
class_df = pd.read_csv("/content/drive/MyDrive/NLP/final_classification.csv")
class_df['tags_list'] = class_df['tags'].apply(lambda x: x.split("_"))
class_df = class_df.sample(frac=1).reset_index(drop=True)
labels = ['love','life','inspirational','philosophy','humor']
label_string = ",".join(labels)

In [ ]:
# constans and hyper-parameters

seeds = [204, 9, 17]
batch_size = 64
epochs = 10

In [ ]:


def split_data(df, random_seed):

  train_val, test_df = train_test_split(df, test_size=0.2, random_state=random_seed, stratify=df['tags'])
  train_df, val_df = train_test_split(train_val, test_size=0.15, random_state=random_seed, stratify=train_val['tags'])

  for d in [train_df, val_df, test_df]:
    tag_count = {}
    for tag in d['tags_list']:
      for t in tag:
        tag_count[t] = tag_count.get(t, 0) + 1
    print(tag_count)

  return train_df, val_df, test_df

In [6]:
def generate_one_hot_encoding(labels, texts):
    one_hot_encodings = []

    for text in texts:
        one_hot_encoding = np.zeros(len(labels), dtype=int)

        for i, label in enumerate(labels):
            if label in text:
                one_hot_encoding[i] = 1

        one_hot_encodings.append(one_hot_encoding)

    return np.array(one_hot_encodings)

In [ ]:
# created dataset class for classification

class ClassificationDataset(Dataset):
    def __init__(self, df, tokenizer):
        super(ClassificationDataset, self).__init__()
        df['quote'] = "classify the quote ("+label_string+"):" + df['quote']
        self.texts = df['quote'].tolist()
        self.targets = df['tags'].tolist()
        self.tokenizer = tokenizer
        self.src_max_length = 512 # based on longest quote
        self.tgt_max_length = 30

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        text = str(self.texts[idx])
        target = str(self.targets[idx])

        encoding = self.tokenizer.encode_plus(
            text,
            max_length=self.src_max_length,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors="pt"
        )
        target_enc = self.tokenizer.encode_plus(
            target,
            max_length=self.tgt_max_length,
            padding="max_length",
            truncation=True,
            return_attention_mask=True,
            return_token_type_ids=False,
            return_tensors="pt"
        )
        input_ids = encoding["input_ids"].squeeze()
        attention_mask = encoding["attention_mask"].squeeze()

        return {
            "input_ids": input_ids.long(),
            "attention_mask": attention_mask.long(),
            "target_ids": target_enc["input_ids"].squeeze().long(),
            "target_attn_mask": target_enc["attention_mask"].squeeze().long()
        }

In [ ]:
def train(model,tokenizer, train_dl, val_dl ,criterion, optimizer, scheduler, epochs):
    nv = 1
    temp = len(train_dl) // nv
    temp = temp - (temp % 100)
    validate_at_steps = [temp * x for x in range(1, nv + 1)]

    train_loss = 0
    for step, batch in enumerate(tqdm(train_dl,
                                      desc='Epoch ' + str(epochs))):
        # set model.eval() every time during training
        model.train()

        # unpack the batch contents and push them to the device (cuda or cpu).
        b_src_input_ids = batch['input_ids'].to(device)
        b_src_attention_mask = batch['attention_mask'].to(device)

        lm_labels = batch['target_ids'].to(device)
        lm_labels[lm_labels[:, :] == tokenizer.pad_token_id] = -100
        b_tgt_attention_mask = batch['target_attn_mask'].to(device)
        optimizer.zero_grad()
        outputs = model(input_ids=b_src_input_ids,
                        attention_mask=b_src_attention_mask,
                        labels=lm_labels,
                        decoder_attention_mask=b_tgt_attention_mask)
        loss = outputs[0]
        train_loss += loss.item()
        loss.backward()
        optimizer.step()
        scheduler.step()
        if step in validate_at_steps:
            print(f'-- Step: {step}')
            _ = val(model, tokenizer,  val_dl, criterion)


    avg_train_loss = train_loss / len(train_dl)
    print('Training loss:', avg_train_loss)

In [ ]:
def val(model,tokenizer, val_dl, criterion):

    val_loss = 0
    true, pred = [], []

    # set model.eval() every time during evaluation
    model.eval()

    for step, batch in enumerate(val_dl):
        # unpack the batch contents and push them to the device (cuda or cpu).
        b_src_input_ids = batch['input_ids'].to(device)
        b_src_attention_mask = batch['attention_mask'].to(device)

        b_tgt_input_ids = batch['target_ids'].to(device)
        lm_labels = b_tgt_input_ids
        #lm_labels[lm_labels[:, :] == tokenizer.pad_token_id] = -100

        b_tgt_attention_mask = batch['target_attn_mask'].to(device)

        # using torch.no_grad() during validation/inference is faster -
        # - since it does not update gradients.
        with torch.no_grad():
            # forward pass
            outputs = model(
                input_ids=b_src_input_ids,
                attention_mask=b_src_attention_mask,
                labels=lm_labels,
                decoder_attention_mask=b_tgt_attention_mask)
            loss = outputs[0]

            val_loss += loss.item()

            # get true
            for true_id in b_tgt_input_ids:
                try:
                  true_decoded = tokenizer.decode(true_id)
                except:
                  # print(b_tgt_input_ids)
                  #print(true_id)
                  true_decoded = ""
                true.append(true_decoded)

            # get pred (decoder generated textual label ids)
            pred_ids = model.generate(
                input_ids=b_src_input_ids,
                attention_mask=b_src_attention_mask
            )
            pred_ids = pred_ids.cpu().numpy()
            for pred_id in pred_ids:
                pred_decoded = tokenizer.decode(pred_id)
                pred.append(pred_decoded)

    true_ohe = generate_one_hot_encoding(labels, true)
    pred_ohe = generate_one_hot_encoding(labels, pred)
    avg_val_loss = val_loss / len(val_dl)
    print('Val loss:', avg_val_loss)
    print('Val Jaccard Score:', jaccard_score(true_ohe, pred_ohe, average="samples"))

    val_micro_f1_score = f1_score(true_ohe, pred_ohe, average='micro')
    print('Val micro f1 score:', val_micro_f1_score)
    return val_micro_f1_score


In [ ]:
def run(model, tokenizer, train_dataloader, validation_dataloader, epochs, seed):
    torch.manual_seed(seed)

    criterion = nn.BCEWithLogitsLoss()

    param_optimizer = list(model.named_parameters())
    no_decay = ["bias", "LayerNorm.bias", "LayerNorm.weight"]
    optimizer_parameters = [
        {
            "params": [
                p for n, p in param_optimizer if not any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.001,
        },
        {
            "params": [
                p for n, p in param_optimizer if any(nd in n for nd in no_decay)
            ],
            "weight_decay": 0.0,
        },
    ]
    optimizer = optim.AdamW(optimizer_parameters, lr=3e-4)

    num_training_steps = len(train_dataloader) * epochs
    scheduler = get_linear_schedule_with_warmup(
        optimizer,
        num_warmup_steps=0,
        num_training_steps=num_training_steps
    )

    max_val_micro_f1_score = float('-inf')
    for epoch in range(epochs):
        train(model,tokenizer, train_dataloader, validation_dataloader, criterion, optimizer, scheduler, epoch)
        val_micro_f1_score = val(model,tokenizer, validation_dataloader, criterion)

        if True:
            if val_micro_f1_score > max_val_micro_f1_score:
                best_model = copy.deepcopy(model)
                best_val_micro_f1_score = val_micro_f1_score

                #model_name = 't5_best_model'
                #torch.save(best_model.state_dict(), model_name + '.pt')

                print(f'--- Best Model. Val loss: {max_val_micro_f1_score} -> {val_micro_f1_score}')
                max_val_micro_f1_score = val_micro_f1_score

    return best_model, best_val_micro_f1_score

In [ ]:

torch.cuda.empty_cache()
torch.cuda.memory_summary(device=None, abbreviated=False)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


In [ ]:

def predict(model, tokenizer,  test_dl):
    val_loss = 0
    pred = []
    model.eval()
    for step, batch in tqdm(enumerate(test_dl), total=len(test_dl)):
        b_src_input_ids = batch['input_ids'].to(device)
        b_src_attention_mask = batch['attention_mask'].to(device)

        with torch.no_grad():
            # get pred
            pred_ids = model.generate(
                input_ids=b_src_input_ids,
                attention_mask=b_src_attention_mask
            )
            pred_ids = pred_ids.cpu().numpy()
            for pred_id in pred_ids:
                pred_decoded = tokenizer.decode(pred_id)
                pred.append(pred_decoded)
    pred_ohe = generate_one_hot_encoding(labels,pred)
    return pred_ohe, pred


In [ ]:
def get_output_tags(out):
  tags = []
  for i in labels:
    if i in out:
      tags.append(i)
  return tags


In [ ]:
def execute_n(df, epochs, seeds, batch_size):
  for seed in seeds:
    # init models and datasets
    model = T5ForConditionalGeneration.from_pretrained("t5-small")
    model = model.to(device)
    tokenizer = T5Tokenizer.from_pretrained("t5-small")

    train_df, val_df, test_df = split_data(df, seed)
    train_data = ClassificationDataset(train_df, tokenizer)
    val_data = ClassificationDataset(val_df, tokenizer)
    train_dataloader = DataLoader(train_data, batch_size=batch_size)
    validation_dataloader = DataLoader(val_data, batch_size=batch_size)
    test_data = ClassificationDataset(test_df, tokenizer)
    test_dataloader = DataLoader(test_data, batch_size=batch_size)

    # train
    t_model, best_f1 = run(model, tokenizer, train_dataloader, validation_dataloader, epochs, seed)

    #test
    test_pred_ohe, pred_labels = predict(t_model, tokenizer, test_dataloader)
    test_df['pred_tags_raw'] = pred_labels
    test_df['pred_tags'] = test_df['pred_tags_raw'].apply(get_output_tags)
    true_ohe = generate_one_hot_encoding(labels, test_df['tags'].tolist())

    jaccard_sim  = jaccard_score(true_ohe, test_pred_ohe, average="samples")

    print(f"Seed: {seed}, Jaccard: {jaccard_sim}")

    #save results
    torch.save(t_model.state_dict(), '/content/drive/MyDrive/NLP/'+str(seed)+'-t5-ft' + '.pt')
    test_df.to_csv('/content/drive/MyDrive/NLP/test_out_'+str(seed)+'.csv', index=False)




In [ ]:
execute_n(class_df, epochs, seeds, batch_size)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have

{'inspirational': 8161, 'philosophy': 6518, 'life': 7696, 'love': 7315, 'humor': 4613}
{'love': 1292, 'life': 1359, 'humor': 815, 'philosophy': 1150, 'inspirational': 1440}
{'love': 2151, 'life': 2264, 'inspirational': 2400, 'philosophy': 1916, 'humor': 1357}


Epoch 0:   0%|          | 0/319 [00:00<?, ?it/s]

-- Step: 300


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Val loss: 22.92721152723881
Val Jaccard Score: 0.60906733351857
Val micro f1 score: 0.6828919112383679


  0%|          | 0/94 [00:00<?, ?it/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


Seed: 17, Jaccard: 0.6072008226335389


,quote,tags,tags_list,pred_tags_raw,pred_tags
0,"classify the quote (love,life,inspirational,ph...",love_life_inspirational_philosophy_humor,"['love', 'life', 'inspirational', 'philosophy'...",<pad> life_inspirational</s><pad><pad><pad><pa...,"['life', 'inspirational']"
1,"classify the quote (love,life,inspirational,ph...",life,['life'],<pad> life</s><pad><pad><pad><pad><pad><pad><p...,['life']
2,"classify the quote (love,life,inspirational,ph...",inspirational_philosophy,"['inspirational', 'philosophy']",<pad> inspirational</s><pad><pad><pad><pad><pa...,['inspirational']
3,"classify the quote (love,life,inspirational,ph...",love_life_inspirational,"['love', 'life', 'inspirational']",<pad> love</s><pad><pad><pad><pad><pad><pad><p...,['love']
4,"classify the quote (love,life,inspirational,ph...",love_life_inspirational_philosophy_humor,"['love', 'life', 'inspirational', 'philosophy'...",<pad> love_life_inspirational_philosophy_humor...,"['love', 'life', 'inspirational', 'philosophy'..."
...,...,...,...,...,...
5992,"classify the quote (love,life,inspirational,ph...",inspirational,['inspirational'],<pad> life</s><pad><pad><pad><pad><pad><pad><p...,['life']
5993,"classify the quote (love,life,inspirational,ph...",love_life_inspirational_philosophy,"['love', 'life', 'inspirational', 'philosophy']",<pad> love_life_inspirational_philosophy</s><p...,"['love', 'life', 'inspirational', 'philosophy']"
5994,"classify the quote (love,life,inspirational,ph...",love_life,"['love', 'life']",<pad> love_life_inspirational_philosophy</s><p...,"['love', 'life', 'inspirational', 'philosophy']"
5995,"classify the quote (love,life,inspirational,ph...",love_life,"['love', 'life']",<pad> humor</s><pad><pad><pad><pad><pad><pad><...,['humor']


0.6570312934244901